# Linear Regression
## Question 1
Make a class called LinearRegression which provides two functions : fit and predict. Try to implement it from scratch. If stuck, refer to the examples folder.

In [ ]:
import numpy as np

class LinearRegressionScratch:
    def __init__(self, learning_rate=0.01, epochs=1000):
        self.lr = learning_rate
        self.epochs = epochs
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y).reshape(-1, 1)
        n_samples, n_features = X.shape

        self.weights = np.zeros((n_features, 1))
        self.bias = 0.0

        for _ in range(self.epochs):
            y_pred = X.dot(self.weights) + self.bias        # (n_samples,1)
            error = y_pred - y

            dw = (1 / n_samples) * X.T.dot(error)
            db = (1 / n_samples) * np.sum(error)

            self.weights -= self.lr * dw
            self.bias    -= self.lr * db

    def predict(self, X):
        X = np.array(X)
        return (X.dot(self.weights) + self.bias).ravel()


## Question 2

Use the dataset https://www.kaggle.com/datasets/quantbruce/real-estate-price-prediction (*).
1. Read it using pandas.
2. Check for **null values**.
3. For each of the columns (except the first and last), plot the column values in the X-axis against the last column of prices in the Y-axis.
4. Remove the unwanted columns.
5. Split the dataset into train and test data. Test data size = 25% of total dataset.
6. **Normalize** the X_train and X_test using MinMaxScaler from sklearn.preprocessing.
7. Fit the training data into the model created in question 1 and predict the testing data.
8. Use **mean square error and R<sup>2</sup>** from sklearn.metrics as evaluation criterias.
9. Fit the training data into the models of the same name provided by sklearn.linear_model and evaluate the predictions using MSE and R<sup>2</sup>.
10. Tune the hyperparameters of your models (learning rate, epochs) to achieve losses close to that of the sklearn models.

Note : (*) To solve this question, you may proceed in any of the following ways :
1. Prepare the notebook in Kaggle, download it and submit it separately with the other questions.
2. Download the dataset from kaggle. Upload it to the session storage in Colab.
3. Use Colab data directly in Colab. [Refer here](https://www.kaggle.com/general/74235). For this, you need to create kaggle API token. Before submitting, hide or remove the API token.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression as SkLinearReg

df = pd.read_csv("real_estate.csv")

print(df.isnull().sum())

cols = df.columns
target_col = cols[-1]
for col in cols[1:-1]:
    plt.figure()
    plt.scatter(df[col], df[target_col])
    plt.xlabel(col)
    plt.ylabel(target_col)
    plt.title(f"{col} vs {target_col}")
    plt.show()


X = df.iloc[:, 1:-1].values   
y = df.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lin_scratch = LinearRegressionScratch(learning_rate=0.05, epochs=5000)
lin_scratch.fit(X_train_scaled, y_train)
y_pred_scratch = lin_scratch.predict(X_test_scaled)

mse_scratch = mean_squared_error(y_test, y_pred_scratch)
r2_scratch = r2_score(y_test, y_pred_scratch)
print("Scratch Linear Regression MSE:", mse_scratch)
print("Scratch Linear Regression R2:", r2_scratch)

sk_lin = SkLinearReg()
sk_lin.fit(X_train_scaled, y_train)
y_pred_sk = sk_lin.predict(X_test_scaled)

mse_sk = mean_squared_error(y_test, y_pred_sk)
r2_sk = r2_score(y_test, y_pred_sk)
print("Sklearn Linear Regression MSE:", mse_sk)
print("Sklearn Linear Regression R2:", r2_sk)


# Logistic Regression
## Question 3

The breast cancer dataset is a binary classification dataset commonly used in machine learning tasks. It is available in scikit-learn (sklearn) as part of its datasets module.
Here is an explanation of the breast cancer dataset's components:

* Features (X):

 * The breast cancer dataset consists of 30 numeric features representing different characteristics of the FNA images. These features include mean, standard error, and worst (largest) values of various attributes such as radius, texture, smoothness, compactness, concavity, symmetry, fractal dimension, etc.

* Target (y):

 * The breast cancer dataset is a binary classification problem, and the target variable (y) represents the diagnosis of the breast mass. It contains two classes:
    * 0: Represents a malignant (cancerous) tumor.
    * 1: Represents a benign (non-cancerous) tumor.

Complete the code given below in place of the "..."

1. Load the dataset from sklearn.datasets
2. Separate out the X and Y columns.
3. Normalize the X data using MinMaxScaler or StandardScaler.
4. Create a train-test-split. Take any suitable test size.

In [ ]:

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression as SkLogReg
import numpy as np

data = load_breast_cancer()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_derivative(z):
    s = sigmoid(z)
    return s * (1 - s)

class LogisticRegressionScratch:
    def __init__(self, learning_rate=0.01, epochs=3000):
        self.lr = learning_rate
        self.epochs = epochs
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y).reshape(-1, 1)
        n_samples, n_features = X.shape

        self.weights = np.zeros((n_features, 1))
        self.bias = 0.0

        for _ in range(self.epochs):
            z = X.dot(self.weights) + self.bias
            y_pred = sigmoid(z)

            error = y_pred - y
            dw = (1 / n_samples) * X.T.dot(error)
            db = (1 / n_samples) * np.sum(error)

            self.weights -= self.lr * dw
            self.bias    -= self.lr * db

    def predict_proba(self, X):
        X = np.array(X)
        z = X.dot(self.weights) + self.bias
        return sigmoid(z).ravel()

    def predict(self, X):
        probs = self.predict_proba(X)
        return (probs >= 0.5).astype(int)


5. Write code for the sigmoid function and Logistic regression.


In [ ]:
def sigmoid(z):
   "..."

def sigmoid_derivative(z):
    "..."

class LogisticRegression:
    def __init__(self, learning_rate, epochs):
      #Initialise the hyperparameters of the model
        self.lr = "..."
        self.epochs = "..."

    def fit(self, X, y):
        n_samples, n_features = X.shape
        y = y.reshape(-1, 1)
        self.weights = "..."
        self.bias = "..."

        #Implement the GD algortihm
        for _ in range(self.epochs):
            z = "..."
            y_pred = "..."

            dw = "..."
            db = "..."

            self.weights -= "..."
            self.bias -= "..."

    def predict(self, X):
      #Write the predict function
        "..."
        return y_pred

6. Fit your model on the dataset and make predictions.
7. Compare your model with the Sklearn Logistic Regression model. Try out all the different penalties.
8. Print accuracy_score in each case using sklearn.metrics .

In [ ]:

scratch_log = LogisticRegressionScratch(learning_rate=0.01, epochs=5000)
scratch_log.fit(X_train, y_train)
y_pred_scratch = scratch_log.predict(X_test)

print("Scratch accuracy:", accuracy_score(y_test, y_pred_scratch))

sk_log_l2 = SkLogReg(max_iter=10000, penalty="l2", solver="lbfgs")
sk_log_l2.fit(X_train, y_train)
y_pred_sk_l2 = sk_log_l2.predict(X_test)
print("Sklearn L2 accuracy:", accuracy_score(y_test, y_pred_sk_l2))

sk_log_l1 = SkLogReg(max_iter=10000, penalty="l1", solver="liblinear")
sk_log_l1.fit(X_train, y_train)
y_pred_sk_l1 = sk_log_l1.predict(X_test)
print("Sklearn L1 accuracy:", accuracy_score(y_test, y_pred_sk_l1))

sk_log_el = SkLogReg(
    max_iter=10000, penalty="elasticnet", solver="saga", l1_ratio=0.5
)
sk_log_el.fit(X_train, y_train)
y_pred_sk_el = sk_log_el.predict(X_test)
print("Sklearn ElasticNet accuracy:", accuracy_score(y_test, y_pred_sk_el))

9. For the best model in each case (yours and scikit-learn), print the classification_report using sklearn.metrics .
10. For the best model in each case (yours and scikit-learn), print the confusion_matrix using sklearn.metrics .

In [ ]:

print("\nScratch classification report:")
print(classification_report(y_test, y_pred_scratch))
print("Scratch confusion matrix:")
print(confusion_matrix(y_test, y_pred_scratch))

print("\nBest sklearn (choose one, e.g., L2) classification report:")
print(classification_report(y_test, y_pred_sk_l2))
print("Best sklearn confusion matrix:")
print(confusion_matrix(y_test, y_pred_sk_l2))


# KNN
## Question 4

How accurately can a K-Nearest Neighbors (KNN) model classify different types of glass based on a glass classification dataset consisting of 214 samples and 7 classes? Use the kaggle dataset "https://www.kaggle.com/datasets/uciml/glass".

Context: This is a Glass Identification Data Set from UCI. It contains 10 attributes including id. The response is glass type(discrete 7 values)

1. Load the data as you did in the 2nd question.
2. Extract the X and Y columns.
3. Split it into training and testing datasets.

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

glass = pd.read_csv("glass.csv")

print(glass.head())
print(glass.isnull().sum())

X = glass.iloc[:, 1:-1].values  
y = glass.iloc[:, -1].values    

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

4. Define Euclidean distance.
5. Build the KNN model.
6. Fit the model on the training data. (Note : you may require to change the type of the data from pandas dataframe to numpy arrays. To do that, just do this X=np.array(X) and so on...)

In [ ]:
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))

class KNNScratch:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X, y):
        self.X_train = np.array(X)
        self.y_train = np.array(y)

    def predict_one(self, x):
        distances = [euclidean_distance(x, x_train) for x_train in self.X_train]
        k_idx = np.argsort(distances)[:self.k]
        k_neighbor_labels = self.y_train[k_idx]
        most_common = Counter(k_neighbor_labels).most_common(1)[0][0]
        return most_common

    def predict(self, X):
        X = np.array(X)
        return np.array([self.predict_one(x) for x in X])

for k in [1, 3, 5, 7]:
    knn_scratch = KNNScratch(k=k)
    knn_scratch.fit(X_train, y_train)
    y_pred = knn_scratch.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"Scratch KNN (k={k}) accuracy:", acc)


7. Make predictions. Find their accuracy using accuracy_score. Try different k values. k=3 worked well in our case.
8. Compare with the sklearn model (from sklearn.neighbors import KNeighborsClassifier)

In [ ]:

for k in [1, 3, 5, 7]:
    sk_knn = KNeighborsClassifier(n_neighbors=k)
    sk_knn.fit(X_train, y_train)
    y_pred_sk = sk_knn.predict(X_test)
    acc_sk = accuracy_score(y_test, y_pred_sk)
    print(f"Sklearn KNN (k={k}) accuracy:", acc_sk)
